# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.


You need to perform random assignment of the data points to clusters (step 1). Next, you should compute the centroids (step 2), and then iterate the above by reassigning labels (step 3) and recomputing means (step 4) until convergence.

#### b) Create the builder function for step 1.

In [1]:
label_randomly <- function(n_points, n_clusters){
  sample(1:n_clusters, n_points, replace = TRUE)
}

#### c) Create the builder function for step 2.

In [2]:
get_cluster_means <- function(data, labels){
    data %>%
      mutate(label__ = labels) %>%
      group_by(label__) %>%
      summarise(across(everything(), mean), .groups = "drop") %>%
      mutate(label = label__) %>%
      arrange(label)
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [3]:
assign_cluster <- function(data, means){
  data_matrix <- as.matrix(data)
  means_matrix <- as.matrix(means %>% select(-label__, -label))

  dists <- outer(rowSums(data_matrix^2), rowSums(means_matrix^2), "+") - 2 * (data_matrix %*% t(means_matrix))

  means$label__[max.col(-dists)]
}

#### e) Create the builder function for step 4.

In [4]:
kmeans_done <- function(old_means, new_means, eps=1e-6){
    om <- old_means %>% select(-label__, -label) %>% as.matrix()
    nm <- new_means %>% select(-label__, -label) %>% as.matrix()

    m <- mean(sqrt(rowSums((om-nm)^2)))
    if(m < eps) TRUE else FALSE
}

#### f) Combine them all into your own `kmeans` function.

In [5]:
mykmeans <- function(data, n_clusters, eps=1e-6){
    labels <- label_randomly(nrow(data), n_clusters)
    old_means <- get_cluster_means(data, labels);
    done <- FALSE
    while (!done){
        labels <- assign_cluster(data, old_means)
        new_means <- get_cluster_means(data, labels)
        if (kmeans_done(old_means, new_means)){
            done <- TRUE
        }

        old_means <- new_means
    }

    list(labels=labels, means=new_means)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set.

In [6]:
library(dplyr)
library(ggplot2)
voltages_df <- read.csv("voltages_df.csv")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`.

In [7]:
results <- mykmeans(voltages_df, 3)
results$labels
results$means

[1] 1 2 2 2 2 2 3 3 2 1 1 1 3 1 2 1 1 1 1 1 2 2 2 2 2 1 1 1 3 3 3 3 2 3 3 2 2
 [38] 1 3 2 1 1 1 3 2 3 3 2 3 3 2 3 1 1 3 1 2 1 3 2 1 1 2 1 3 3 2 3 1 3 1 2 3 3
 [75] 2 2 1 3 3 3 3 3 2 3 3 2 2 2 1 3 3 3 2 3 2 2 3 2 2 3 3 3 2 3 1 1 3 1 1 2 1
[112] 1 2 3 1 3 1 1 2 2 2 2 1 1 3 2 2 3 1 3 2 1 2 3 1 2 2 2 1 3 1 2 2 3 1 1 2 1
[149] 1 3 1 3 2 2 3 1 3 3 2 1 3 3 1 2 1 3 2 1 2 1 1 3 2 1 3 3 2 3 1 2 3 2 3 1 3
[186] 2 2 1 2 1 1 2 1 1 3 3 1 2 1 1 3 1 3 3 2 1 2 3 2 1 2 3 1 3 3 2 3 3 2 2 2 2
[223] 2 2 2 3 3 3 3 1 2 1 3 2 2 3 3 1 1 3 3 1 3 3 2 3 1 3 3 2 3 2 1 3 2 3 3 1 1
[260] 1 2 2 3 1 2 2 2 3 3 3 1 2 2 2 3 1 3 3 2 2 3 1 3 1 1 2 3 2 2 3 3 1 3 3 1 2
[297] 1 3 3 2 2 3 1 1 1 3 1 2 2 2 1 1 1 1 1 1 2 1 2 1 1 3 2 3 1 1 1 1 2 1 2 1 3
[334] 1 2 2 3 3 1 2 3 2 3 3 1 3 2 3 1 1 3 2 2 2 1 3 2 1 3 3 2 2 2 2 1 3 1 2 1 1
[371] 1 1 1 2 2 3 3 1 1 1 1 3 3 1 1 3 3 2 2 3 1 2 2 2 2 3 2 2 3 1 1 1 1 2 2 1 1
[408] 3 3 3 2 3 3 1 2 2 2 3 2 2 2 1 1 2 1 3 2 2 2 2 3 3 2 3 1 2 1 1 1 3 3 3 3 1
[445] 2 1 1 1 3 2 1 1 1 2 2 1 2 2 1 3 2 2 1 3 2 2 2 1 3 3 1 3 1 2 2 3 1 3 1 2 2
[482] 3 1 3 3 3 2 3 3 3 3 3 2 1 1 3 3 1 2 2 2 1 1 1 2 2 3 3 1 3 1 2 2 3 3 1 3 3
[519] 2 1 1 1 2 3 1 2 3 3 1 2 3 3 3 3 1 1 2 1 1 2 2 2 2 3 1 1 3 1 1 1 2 2 1 2 3
[556] 3 2 3 3 3 2 3 3 2 3 3 2 3 3 1 3 2 3 2 3 2 2 2 1 3 3 3 3 1 2 3 3 3 2 3 3 2
[593] 2 2 3 1 2 2 1 2 3 2 2 3 1 1 3 3 3 2 3 1 2 1 3 2 1 2 2 1 3 2 1 3 3 1 2 1 3
[630] 2 1 3 1 2 2 1 2 1 1 1 1 1 2 2 1 3 1 3 3 1 3 2 3 1 3 1 1 1 2 1 1 2 2 1 1 2
[667] 1 2 3 3 2 1 2 1 1 3 2 2 1 3 1 2 3 3 1 1 2 1 2 2 3 3 3 3 2 2 2 1 3 2 1 3 3
[704] 1 1 3 2 3 3 1 2 1 3 2 1 2 2 1 2 2 1 1 2 2 1 2 3 1 1 1 1 1 2 2 1 1 2 2 1 2
[741] 1 2 3 1 1 1 2 1 3 2 2 1 1 3 1 1 3 1 3 3 1 1 2 3 1 2 3 2 2 1 3 2 3 1 2 1 1
[778] 3 1 3 2 1 1 2 3 2 1 2 1 3 1 1 2 2 2 1 2 1 3 2 2 2 2 1 1 3 2 1 1 1 1 3 3 1
[815] 3 1 2 1 2 3 2 2 2 2 2 3 3 2 3 3 3 3 3 3 2 1 1 2 3 3 1 2 3 3 3 3 1 1 2 3 2
[852] 1 3 3 3 3 1 1 3 3 3 3 3 1 2 3 2 1 2 2 3 1 3 3 1 1 2 1 1 1 3 2 3 2 2 1 1 3
[889] 3 2 1 3 3 2 2 2 2 1 3 3

label__,X0,X1.00401606425703,X2.00803212851406,X3.01204819277108,X4.01606425702811,X5.02008032128514,X6.02409638554217,X7.0281124497992,X8.03212851405623,⋯,X241.967871485944,X242.971887550201,X243.975903614458,X244.979919678715,X245.983935742972,X246.987951807229,X247.991967871486,X248.995983935743,X250,label
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,-1.031463,0.9381238,0.7619864,0.3631543,-1.1179412,-1.051145,-0.9766807,-0.8694758,-0.6892375,⋯,-0.8070676,-0.8182598,-0.8207339,-0.8132928,-0.7969549,-0.77567272,-0.75689256,-0.7496483,-0.7570393,1
2,-1.031463,1.2439759,1.0924697,0.9004440,0.3011754,-1.159714,-1.1098127,-1.0685484,-1.0338649,⋯,-0.8732292,-0.8234477,-0.7607812,-0.6682618,-0.3380864,-0.04693168,0.02820486,-0.4113500,-0.8115784,2
3,-1.031463,1.3093239,1.1616772,0.9787498,0.6481497,-1.168610,-1.1196122,-1.0590962,-0.9943176,⋯,0.8337474,0.7125412,-0.2659209,-1.0409179,-1.0587745,-1.01359887,-0.96467777,-0.9151047,-0.8610245,3


#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`.
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [8]:
voltages_matrix <- as.matrix(voltages_df)
kmeans_function_result <- kmeans(voltages_matrix, centers = 3)
kmeans_function_result$centers
kmeans_function_result$cluster

,X0,X1.00401606425703,X2.00803212851406,X3.01204819277108,X4.01606425702811,X5.02008032128514,X6.02409638554217,X7.0281124497992,X8.03212851405623,X9.03614457831325,⋯,X240.963855421687,X241.967871485944,X242.971887550201,X243.975903614458,X244.979919678715,X245.983935742972,X246.987951807229,X247.991967871486,X248.995983935743,X250
1,-1.031463,1.309623,1.1620520,0.9792874,0.6451702,-1.170639,-1.119646,-1.0589724,-0.9939507,-0.9229994,⋯,0.2225596,0.7762542,0.7248267,-0.1676761,-0.9950115,-1.0490881,-1.0146832,-0.9660534,-0.9166152,-0.8626095
2,-1.031463,1.091050,0.9272281,0.6317991,-0.4083829,-1.105430,-1.043247,-0.9690121,-0.8615512,-0.7841859,⋯,-0.8503929,-0.8401484,-0.8208538,-0.7907575,-0.7407773,-0.5675206,-0.4113022,-0.3643438,-0.5804991,-0.7843089
3,-1.031463,1.309127,1.1614308,0.9783963,0.6501085,-1.167275,-1.119590,-1.0591777,-0.9945589,-0.9242331,⋯,0.4112895,0.8715468,0.7044639,-0.3305127,-1.0710994,-1.0651429,-1.0128860,-0.9637734,-0.9141117,-0.8599824


[1] 2 2 2 2 2 2 1 1 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 1 3 2 1 3 2 2
 [38] 2 3 2 2 2 2 1 2 3 1 2 3 1 2 3 2 2 1 2 2 2 3 2 2 2 2 2 1 3 2 3 2 3 2 2 1 3
 [75] 2 2 2 1 3 1 1 3 2 3 3 2 2 2 2 3 3 3 2 3 2 2 3 2 2 3 3 3 2 1 2 2 3 2 2 2 2
[112] 2 2 1 2 3 2 2 2 2 2 2 2 2 3 2 2 3 2 3 2 2 2 3 2 2 2 2 2 1 2 2 2 3 2 2 2 2
[149] 2 3 2 3 2 2 1 2 3 1 2 2 3 3 2 2 2 3 2 2 2 2 2 1 2 2 1 1 2 3 2 2 1 2 1 2 1
[186] 2 2 2 2 2 2 2 2 2 3 3 2 2 2 2 1 2 3 1 2 2 2 3 2 2 2 3 2 3 3 2 3 3 2 2 2 2
[223] 2 2 2 3 1 3 3 2 2 2 3 2 2 3 3 2 2 1 3 2 3 1 2 1 2 3 3 2 3 2 2 3 2 1 3 2 2
[260] 2 2 2 3 2 2 2 2 1 3 1 2 2 2 2 3 2 1 3 2 2 1 2 3 2 2 2 1 2 2 3 3 2 1 3 2 2
[297] 2 1 3 2 2 3 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 3 2 2 2 2 2 2 2 2 1
[334] 2 2 2 1 1 2 2 3 2 3 1 2 1 2 3 2 2 1 2 2 2 2 3 2 2 3 3 2 2 2 2 2 3 2 2 2 2
[371] 2 2 2 2 2 3 3 2 2 2 2 3 3 2 2 3 1 2 2 3 2 2 2 2 2 3 2 2 3 2 2 2 2 2 2 2 2
[408] 3 3 1 2 3 3 2 2 2 2 1 2 2 2 2 2 2 2 3 2 2 2 2 1 3 2 1 2 2 2 2 2 1 3 1 1 2
[445] 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 3 2 2 2 1 2 2 2 2 1 1 2 3 2 2 2 1 2 3 2 2 2
[482] 3 2 3 3 1 2 3 3 3 1 1 2 2 2 3 3 2 2 2 2 2 2 2 2 2 3 3 2 3 2 2 2 1 3 2 3 1
[519] 2 2 2 2 2 3 2 2 3 3 2 2 3 1 3 3 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 3
[556] 1 2 3 1 1 2 3 1 2 3 1 2 1 1 2 3 2 1 2 3 2 2 2 2 3 1 1 3 2 2 3 3 3 2 3 3 2
[593] 2 2 3 2 2 2 2 2 1 2 2 3 2 2 3 3 1 2 1 2 2 2 3 2 2 2 2 2 3 2 2 3 3 2 2 2 1
[630] 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 1 2 1 2 3 2 3 2 2 2 2 2 2 2 2 2 2 2
[667] 2 2 1 3 2 2 2 2 2 3 2 2 2 1 2 2 3 1 2 2 2 2 2 2 3 3 1 3 2 2 2 2 1 2 2 1 1
[704] 2 2 3 2 3 3 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2
[741] 2 2 1 2 2 2 2 2 1 2 2 2 2 3 2 2 3 2 1 3 2 2 2 1 2 2 3 2 2 2 1 2 1 2 2 2 2
[778] 3 2 1 2 2 2 2 3 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2 2 2 3 1 2
[815] 3 2 2 2 2 1 2 2 2 2 2 3 3 2 3 3 1 3 3 1 2 2 2 2 1 3 2 2 1 3 1 1 2 2 2 3 2
[852] 2 1 1 3 3 2 2 3 1 3 1 3 2 2 3 2 2 2 2 3 2 3 3 2 2 2 2 2 2 1 2 1 2 2 2 2 3
[889] 1 2 2 3 1 2 2 2 2 2 3 3

In [9]:
install.packages("gridExtra")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



#### d) Are your labels/clusters the same? If not, why? Are your means the same?

No they are not the same. The labels/clusters change base on the initial assignments which is random, and so the labels the points are assigned to might be different even though the overall pattern is similar. Our means are not exactly the same.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

For the For Loop, you have to loop over every data point, and within it loop over the clusters means to get the distance between the centroid and the point, and obtain the smallest distance.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

In the vectorizing process, you take the centroids and the points and make them into two matrices, then perform matrix operations to obtain the smallest distance.

#### c) State which (for loops or vectorizing) is more efficient and why.

Vectorizing is faster because you can obtain the smallest distance in one step through matrix operations without explicit loops. The double loops approach is more time-consuming.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

Kmeans fail when the clusters are different in size or nonspherical. Kmeans assumes the data is in a vector space, the clusters are characterized by the cluster means and are relatively the same size, and that the distance between points and centroids determine the assignment equally.

## Question 5
#### What assumption do Guassian mixture models make?

Gaussian Mixture Models assume the data comes from a combination of multiple Gaussian distributions. This means the uniformity assumption is relaxed and it allows us to work with clusters of different sizes and shapes.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

Spectral clustering assumes that if two points are close together (pairwise), they are more probable to belong in the same cluster. This is different from k-means because k-means determine clusters based on distance from centroids. Spectral clustering lowers the assumptions required by k-means and allowed for more variety of data that can be used for clustering.

## Question 7
#### Define the gap statistic method. What do we use it for?

The gap statistic method measures the dispersion in both the clustering of our choice, and a "randomized" data, and then calcuates the difference (gap).

It is used to determine the optimal number of clusters (gap is considerably large) and help consider if our clustering is statistically better in performance than at random.